# Getting Started with Jupyter Notebooks

This file is a "Jupyter Notebook". Jupyter Notebooks are files that allow one to write and evaluate Python (and R, and Julia...) alongside documentation, which makes them great for exploratory code investigations.

To run this notebook locally on your machine, we recommend that you follow these steps.

### Installing Anaconda (Optional)

To follow along, the first step will be to [install Anaconda](https://www.anaconda.com/download/), a distribution of the Python programming language that helps make managing Python easier.

Once Anaconda is installed, open a new terminal window. (If you are on Windows, you should open an Anaconda terminal by going to Programs -> Anaconda3 (64-bit) -> Anaconda Prompt). Then you can create and activate a virtual environment:

```
# create a virtual environment with Python 3.6 named "3.6"
conda create python=3.6 --name=3.6

# activate the virtual environment
source activate 3.6
```

### Running the Workshop Notebook

You should now see `(3.6)` prepended on your path. Once you see that prefix, you can start the notebook with the following commands:

```
git clone https://github.com/YaleDHLab/lab-workshops
cd lab-workshops/machine-learning
pip install -r requirements.txt
jupyter notebook machine-learning.ipynb
```

Once the notebook is open, you can evaluate a code cell by clicking on that cell, then clicking `Cell -> Run Cells`. Alternatively, after clicking on a cell you can hold Control and press Enter to execute the code in the cell. To run all the cells in a notebook (which I recommend you do for this notebook), you can click `Cell -> Run All`.

If you want to add a new cell, click the "<b>+</b>" button near the top of the page (below and between File and Edit). In that new cell, you can type Python code, like `import this`, then run the cell and immediately see the output. I encourage you to add and modify cells as we move through the discussion of machine learning below, as interacting with the code is one of the best ways to grow comfortable with the techniques discussed below.

# Introduction to Word Vectors

"Word vectors", or "word embeddings" are ways of representing semantic and syntactic information about words in a vector form. As you may know, the word "vector" is just a fancy term for a list of numbers (or sometimes a list of number lists). Word vectors are a way of representing each word in a vocabulary with a list of numbers, such that those numbers can tell us useful information about the words in the vocabulary.

The simplest kind of word vector system represents each distinct word in a vocabulary with a `w` dimensional vector (or list of `w` numbers), where `w` equals the number of distinct words in the language. Each word's vector consists of zeros for all but the `i-th` value in the list, where `i` indicates the given word's index position within the vocabulary. This is known as a "[one-hot encoding](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f)," as the vector contains 0s in all but one position.

For example, suppose one has a vocabulary consisting only of five words: King, Queen, Man, Woman, and Child. In that case one could encode the word "Queen" as:

<img src='images/word2vec-one-hot.png' style='width: 600px'>

<div style='text-align: center'><a href='https://blog.acolyer.org/2016/04/21/the-amazing-power-of-word-vectors/'>Image by Adrian Coyler</a></div>

While it's easy to represent words with this kind of one-hot encoding, these vectors don't give us any way to compare words except to check if they're equal, which isn't very helpful.

Recent approaches to word vectors, including Google's Word2Vec and Stanford University's GloVe embeddings, create more insightful word vectors by representing each word in a language with a dense `k` dimensional vector, where `k` is a value chosen by the user who creates the word vectors. (Dense vectors are lists of numbers in which there are very few if any zeros; unlike one-hot vectors which are comprised of all zeros with a single 1, dense vectors are comprised almost entirely of non-zero values). The meaning of each word is thereby represented by a list of `k` values, and each unit in the `k` dimensions contributes some meaning to each word.

If one were to label the dimensions in a word vector, the result might look something like the following:

<img src='images/word2vec-distributed-representation.png' style='width: 600px'>

<div style='text-align: center'><a href='https://blog.acolyer.org/2016/04/21/the-amazing-power-of-word-vectors/'>Image by Adrian Coyler</a></div>

Each of these word vectors (the columns above) gives a representation of the semantic and syntactic function of a word in a language. By comparing these vectors, we can study relationships between words in ways that were previously not possible. To see how this works, let's dive into some code below.

# Preparing the Code

The following block loads some dependencies, clears all warnings, and makes random number generation reproducible.

In [ ]:
%load_ext autoreload
%autoreload 2 # reimport modules when evaluating cells

import warnings
warnings.filterwarnings('ignore') # ignore all warnings

import numpy as np
np.random.seed(0) # make random number generation consistent

# Loading Pretrained Word Vectors

The following section will allow us to get started with word vectors by loading a pretrained model trained by Google. This model has already learned the mapping from each word to a 300 dimensional vector, so we won't need to "train" this model.

In [ ]:
import requests
import shutil

# step one: download (1.5GB) Google's model to your current directory
url = 'https://s3.amazonaws.com/lab-data-collections/GoogleNews-vectors-negative300.bin.gz'
open('GoogleNews-vectors-negative300.bin.gz', 'wb').write(requests.get(url, allow_redirects=True).content)

In [ ]:
import gzip

# step two: unzip the gzipped model file we just downloaded
with gzip.open('GoogleNews-vectors-negative300.bin.gz', 'rb') as f:
  with open('pretrained-model.bin', 'wb') as out:
    for i in f:
      out.write(i)

In [ ]:
from gensim.models import KeyedVectors

# step three: load the pretrained model we just downloaded and unzipped
model = KeyedVectors.load_word2vec_format('pretrained-model.bin', binary=True)  

# Exploring the Model

Now that we've downloaded and loaded a pretrained model, let's see what this model can do.

To do so, let's use one of the most useful Python commands, `dir()`, which displays all of the attributes (data appended to object) and methods (functions we can call on an object) defined on the model we just instantiated.

In [ ]:
# investigate all attributes on the model
dir(model)

There are two attributes that are of particular interest in this list: `vocab` and `wv`. The former lists all of the words in the model, while the latter lets us fetch the vector associated with a given word:

In [ ]:
print(list(model.vocab)[-100:])

In [ ]:
print(model.wv['tiptop'])

We'll use these attributes below to investigate the model more thoroughly.

# Comparing Word Vectors

Now that we've downloaded and loaded a pretrained model, let's use it to compare how similar words are. We should expect words we associate with one-another to have high similarity values, and words that we don't associate together to have low similarity values.

In [ ]:
from scipy.spatial.distance import cosine
import numpy as np

# get the vectors associated with two sample words
vec_a = model.wv['beach']
vec_b = model.wv['ocean']

# find the n most similar terms to a given query vector
print(model.wv.similar_by_vector(vec_a, topn=3))

# show the distance between those vectors
print(cosine(vec_a, vec_b))

In [ ]:
# if you try to get the vector associated with a word that doesn't exist, an error springs!
word = 'cats_in_trees'

try:
  model.wv[word]
except KeyError:
  print(' ! Word missing from vocabulary!', word)

In [ ]:
# find all words in the model
words = list(model.wv.vocab.keys())

words[:100] # show just the first hundred words in the model

# Visualizing Word Vector Fields

In the code above, we examined the simialarity between two word vectors. In what follows, we'll visualize the similarity between hundreds of words at once. To do so, we'll reduce the vector representation of each word to just two dimensions, then we'll create a visualization that renders each word at its two-dimensional position.

In [ ]:
%matplotlib inline

# visualize the field of terms fed as input
from sklearn.manifold import TSNE
from adjustText import adjust_text
import matplotlib.pyplot as plt
from umap import UMAP
import numpy as np
import operator


def plot_word_field(model, method='umap', skip=0, max_words=1000, jitter=0, margin=0.05, figsize=(20, 14), min_dist=0.1):
  '''
  Given a gensim.model instance, plot the positions of terms
  '''
  if method not in ['tsne', 'umap']: raise Exception(' ! Requested method is not supported')
  words = get_model_words(model, max_words=max_words, skip=skip)
  if not words: raise Exception(' ! No words were found in model -- exiting')
  word_vectors = np.array([model.wv[word] for word in words]) # array of vecs, one per word
  print(' * creating layout for', len(word_vectors), 'words')
  if method == 'umap':
    X = UMAP(n_neighbors=5, min_dist=min_dist).fit_transform(word_vectors) # X.shape = len(words), 2
  elif method == 'tsne':
    X = TSNE(n_components=2).fit_transform(word_vectors) # X.shape = len(words), 2
  plot_words(X, words, figsize=figsize, jitter=jitter, margin=margin)

  
def get_model_words(model, max_words=1000, skip=0):
  '''Get the words from `model`. If `max_words` is provided, return up to that many words'''
  words = list(model.wv.vocab.keys()) # find all words in the model
  if max_words: # get the n most popular words if user requested max words
    word_to_count = {w: model.wv.vocab[w].count for w in words}
    sorted_by_count = sorted(word_to_count.items(), key=operator.itemgetter(1))
    sorted_by_count.reverse()
    words = [i[0] for i in sorted_by_count[skip:skip+max_words]]
  return words


def plot_words(X, words, jitter=False, figsize=(10,6), margin=0.5, labels={}):
  '''Given `X` where shape = words,2 and list of strings `words` plot the words at positions in `X`'''
  plt.figure(figsize=figsize)
  words = [plt.text(X[word_idx][0], X[word_idx][1], word) for word_idx, word in enumerate(words)]
  if jitter: adjust_text(words, lim=jitter)
  if labels.get('x', False): plt.xlabel(labels['x'])
  if labels.get('y', False): plt.ylabel(labels['y'])
  # set the axis ranges
  x_vals = [w.get_position()[0] for w in words]
  y_vals = [w.get_position()[1] for w in words]
  plt.xlim(( min(x_vals)-margin, max(x_vals)+margin ))
  plt.ylim(( min(y_vals)-margin, max(y_vals)+margin ))
  plt.show()


In [ ]:
words = plot_word_field(model, skip=5000, max_words=500, jitter=0, min_dist=0.25, method='umap')

# Finding Approximate Nearest Neighbors

In [ ]:
from annoy import AnnoyIndex
import json
import os


def build_ann_index(model):
  '''Build an ANN model and persist to disk for faster vector similarity queries'''
  words = list(model.wv.vocab.keys()) # list of strings, one per word
  idx_to_word = {str(idx): i for idx, i in enumerate(words)} # d[word] = word_idx in words
  dims = model.wv[words[0]].shape[0] # number of dimensions in each input vector

  # create the approximate nearest neighbors model
  if not os.path.exists('model.ann'):
    ann = AnnoyIndex(dims)
    for i in idx_to_word:
      ann.add_item(int(i), model.wv[idx_to_word[i]])
    ann.build(10) # number of 'trees' to build
    ann.save('model.ann')
  if not os.path.exists('idx_to_word.json'):
    with open('idx_to_word.json', 'w') as out: json.dump(idx_to_word, out)

  # load the saved model
  ann = AnnoyIndex(dims)
  ann.load('model.ann')
  idx_to_word = json.load(open('idx_to_word.json'))
  
  # return the model for querying and the map from word to idx
  return ann, idx_to_word


def find_centroid(words):
  '''Given a list of words, get the centroid of those word's vectors'''
  vecs = np.vstack([model.wv[w] for w in words if w in model.wv])
  sums = np.array([ np.sum(vecs[:,idx]) for idx, i in enumerate(range(vecs[0].shape[0])) ])
  return sums / vecs[0].shape[0]


def find_similar_by_vec(vec, n=50):
  '''Return the words for the `n` most similar words to a query vector'''
  indices = ann.get_nns_by_vector(vec, n*2**2, search_k=-1, include_distances=False)
  similar_words = [idx_to_word[str(i)] for i in indices]
  curated = []
  for idx, word in enumerate(similar_words):
    if len(curated) < n:
      if similar_words[idx].lower() not in curated:
        curated.append(similar_words[idx].lower())
  return curated


def find_similar_by_words(words, n=50):
  '''Return the words for the `n` most similar words to a list of query words'''
  centroid = find_centroid(words)
  indices = ann.get_nns_by_vector(centroid, n*2**2, search_k=-1, include_distances=False)
  similar_words = [idx_to_word[str(i)] for i in indices]
  curated = []
  for idx, word in enumerate(similar_words):
    if len(curated) < n:
      if similar_words[idx].lower() not in words and similar_words[idx].lower() not in curated:
        curated.append(similar_words[idx].lower())
  return curated


# prepare data structures that will expedite the process of finding words similar to a query vector
ann, idx_to_word = build_ann_index(model)

# Finding Terms Related to a Concept

In [ ]:
# create lists of words in a conceptual category        
veggie_words = ['asparagus', 'artichoke', 'avocado', 'beets', 'broccoli', 'carrot', 'celery', 'cauliflower', 'cucumber', 'eggplant', 'kale', 'lentils', 'lettuce', 'mushroom', 'olive', 'onion', 'pea', 'potato', 'salad', 'spinach', 'squash', 'tomato', 'turnip', 'yam', 'zucchini']
meat_words = ['bacon', 'beef', 'chicken', 'crab', 'duck', 'goose', 'meat', 'meatball', 'mutton', 'offal', 'partridge', 'pheasant', 'pork', 'quail', 'rabbit', 'turkey', 'veal', 'venison']

other_veggie_words = find_similar_by_words(veggie_words)
other_meat_words = find_similar_by_words(meat_words)

In [ ]:
other_meat_words

# Plotting relationships between concepts

In [ ]:
from scipy.spatial.distance import cosine as dist

def plot_distances_to_concepts(words, vec_one, vec_two, jitter=0, labels={}):
  '''Plot each item in `words` accordint to its distance from `vec_one` and `vec_two`'''
  # 2D array where subarrays = [dist_from_vec_one, dist_from_vec_two]
  words = [w for w in words if w in model.wv]
  distances = [[1-dist(model.wv[w], vec_one), 1-dist(model.wv[w], vec_two)] for w in words]
  plot_words(np.vstack(distances), words, jitter=jitter, labels=labels) 

# find the centroids for each cluster of words
veggie_centroid = find_centroid(veggie_words)
meat_centroid = find_centroid(meat_words)

# plot the words in the concept space
words = other_veggie_words + other_meat_words
plot_distances_to_concepts(words, veggie_centroid, meat_centroid, jitter=4, labels={'x': 'vegginess', 'y': 'meatiness'})

# Creating Custom Word2Vec Models with Gensim

In [ ]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
import requests as r
import os

def download_text(url, out_dir='texts'):
  '''Download the content at `url` to the present directory'''
  if not os.path.exists(out_dir): os.makedirs(out_dir)
  response = r.get(url, allow_redirects=True)
  out_path = os.path.join(out_dir, os.path.basename(i))
  open(out_path, 'wb').write(response.content)
  print(' * downloaded', url)

# specify the urls to one or more text files
urls = [
  'http://www.gutenberg.org/cache/epub/14/pg14.txt',
  'http://www.gutenberg.org/cache/epub/25/pg25.txt',
  'http://www.gutenberg.org/cache/epub/48/pg48.txt',
]

# download each of the text files specified above
for i in urls: download_text(i)

In [ ]:
import codecs
import glob

# generate a list of lists, where sublists contain words from a file (in order)
word_lists = []
for i in glob.glob('texts/*'):
  with codecs.open(i) as f:
    word_lists.append(f.read().lower().split())

# build a model with the custom word lists
model = Word2Vec(word_lists, size=100, window=5, min_count=20, workers=4)
model.save('word2vec.model')

In [ ]:
# plot the words
plot_word_field(model, method='umap', min_dist=0.9)

# Hierarchical Word Modelling

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage

max_words = 1000
skip = 1000

words = get_model_words(model, max_words=max_words, skip=skip)
vecs = [model.wv[i] for i in words]

Z = linkage(vecs, 'ward')

In [ ]:
fig, ax = plt.subplots(figsize=(max_words/4, 8))
dendrogram(Z, leaf_rotation=90.0, leaf_font_size=14.0, labels=words)
plt.show()